# **Extracting Eyeblink features from EEG**

## **1) Importing libraries and reading EDF file**


In [1]:
import mne
import matplotlib

matplotlib.use('Qt5Agg')

#fname = "sub_1_03.edf"
fname = "1-1.edf"
raw = mne.io.read_raw_cnt(fname, preload=True)

<ipython-input-1-1a3c2a7a37f1>:8: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = mne.io.read_raw_cnt(fname, preload=True)
C:\Users\USER\.conda\envs\mne_new\lib\site-packages\mne\io\cnt\cnt.py:241: RuntimeWarning: overflow encountered in long_scalars
  data_size // (n_samples * n_channels) not in [2, 4]):
<ipython-input-1-1a3c2a7a37f1>:8: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = mne.io.read_raw_cnt(fname, preload=True)
C:\Users\USER\.conda\envs\mne_new\lib\site-packages\mne\io\cnt\cnt.py:282: RuntimeWarning: overflow encountered in float_scalars
  cals.append(cal * sensitivity * 1e-6 / 204.8)


UnicodeDecodeError: 'ascii' codec can't decode byte 0x80 in position 1: ordinal not in range(128)

## **2) Selecting channels of interest & info**

In [42]:
raw_eeg_d = raw.copy().pick_types(meg=False, eeg=True, eog=False, exclude=['P7', 'P4', 'Cz', 'Pz', 'P3', 'P8', 'O1', 'O2', 'T8', 'F8', 'C4', 'F4', 'Fz', 'C3', 'F3', 'T7', 'F7', 'EXT','X','Y','Z'])
len(raw_eeg_d.ch_names)

2

In [2]:
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=False, exclude=['EXT','X','Y','Z'])
len(raw_eeg.ch_names)

19

In [3]:
raw_eeg.info

<Info | 7 non-empty values
 bads: []
 ch_names: P7, P4, Cz, Pz, P3, P8, O1, O2, T8, F8, C4, F4, Fp2, Fz, C3, F3, ...
 chs: 19 EEG
 custom_ref_applied: False
 highpass: 0.1 Hz
 lowpass: 250.0 Hz
 meas_date: 2013-06-23 09:55:25 UTC
 nchan: 19
 projs: []
 sfreq: 500.0 Hz
>

In [3]:
montage_kind = "standard_1020"
montage = mne.channels.make_standard_montage(montage_kind)
raw_eeg.set_montage(montage, match_case=False)

<RawEDF | sub_1_03.edf, 19 x 1260500 (2521.0 s), ~182.8 MB, data loaded>

In [5]:
raw_eeg

<RawEDF | sub_1_03.edf, 19 x 1260500 (2521.0 s), ~182.8 MB, data loaded>

In [6]:
print('Size of the matrix: {}\n'.format(raw_eeg.get_data().shape))

Size of the matrix: (19, 1260500)



## **3) Plots raw**

### a) Plot based on suitable scale

In [23]:
raw_eeg.plot(duration = 20,scalings = 334985)

<MNEBrowseFigure size 1366x705 with 4 Axes>

### b) Plot of PSD

In [11]:
raw_eeg.plot_psd(fmax=150)

Effective window size : 4.096 (s)


<ipython-input-11-d29de30f2258>:1: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  raw_eeg.plot_psd(fmax=150)


<MNELineFigure size 1000x350 with 1 Axes>

## **4) Applying filters**

### a) Apply Bandpass filter

In [4]:
raw_eeg.load_data()
raw_eeg_filt_05_50 = raw_eeg.copy().filter(l_freq=0.5, h_freq=40)
raw_eeg_filt_1_37 = raw_eeg.copy().filter(l_freq=1, h_freq=37)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 37 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband ed

### b) Apply Notch filter (need this?)

In [ ]:
raw_eeg_filt_1_50_notch = raw_eeg_filt_1_50.copy().filter_notch()

### c) Plot filtered data

In [6]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

raw_eeg.plot_psd(ax=ax[0], show=False)
raw_eeg_filt_1_37.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(False) 
# see other options to display 
plt.show()

Effective window size : 4.096 (s)
Effective window size : 4.096 (s)


In [46]:
raw_eeg_filt_1_37.plot(title = 'Filtered data', duration = 20, scalings = 334985)

<MNEBrowseFigure size 1366x705 with 4 Axes>

Channels marked as bad: none
Channels marked as bad: none


## **5) Applying ICA**

In [9]:
eeg_ica = raw_eeg_filt_1_37.copy()

In [37]:
n_components = 19
method = 'picard' #can converge faster than Infomax and FastICA
max_iter = 500 # just to prevent infinite loop - normally 500 or 1000
fit_params = dict(fastica_it=5) #can drop this also
random_state = 10 # fix a number so that can get same results

ica = mne.preprocessing.ICA(n_components=n_components,
                            random_state=random_state,
                            method=method,
                            fit_params=fit_params,
                            max_iter=max_iter)
tempica = ica.fit(eeg_ica)

Fitting ICA to data using 19 channels (please be patient, this may take a while)
Selecting by number: 19 components
Fitting ICA took 112.8s.


In [21]:
ica.plot_components(inst = eeg_ica)

[<Figure size 975x749 with 19 Axes>]

## **6) Saving output as EDF file**

In [ ]:
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif', 
                              overwrite=True)

In [27]:
ica.plot_sources(eeg_ica, show_scrollbars=True)

Creating RawArray with float64 data, n_channels=19, n_times=1260500
    Range : 0 ... 1260499 =      0.000 ...  2520.998 secs
Ready.


<MNEBrowseFigure size 1366x705 with 4 Axes>

In [40]:
tempicadata = ica.get_sources(eeg_ica)

In [44]:
tempicadata.plot(title = 'ica data', duration = 20, scalings = 10)

<MNEBrowseFigure size 1366x705 with 4 Axes>

In [52]:
import csv
  
  
# field names 
fields = ['Name', 'Branch', 'Year', 'CGPA'] 
    
# data rows of csv file 
rows = [ ['Nikhil', 'COE', '2', '9.0'], 
         ['Sanchit', 'COE', '2', '9.1'], 
         ['Aditya', 'IT', '2', '9.3'], 
         ['Sagar', 'SE', '1', '9.5'], 
         ['Prateek', 'MCE', '3', '7.8'], 
         ['Sahil', 'EP', '2', '9.1']] 
  
with open('try.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

In [53]:
tempicadata.info

<Info | 8 non-empty values
 bads: []
 ch_names: ICA000, ICA001, ICA002, ICA003, ICA004, ICA005, ICA006, ICA007, ...
 chs: 19 MISC
 custom_ref_applied: False
 dig: 22 items (3 Cardinal, 19 EEG)
 highpass: 1.0 Hz
 lowpass: 37.0 Hz
 meas_date: 2013-06-23 09:55:25 UTC
 nchan: 19
 projs: []
 sfreq: 500.0 Hz
>

In [57]:
tempicadata.ch_names[1]

'ICA001'

In [58]:
df=tempicadata.to_data_frame()
df.to_csv('file.csv',index=False)

In [59]:
filtf=raw_eeg_filt_1_37.to_data_frame()
filtf.to_csv('fit_eeg.csv',index=False)